In [1]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import monotonically_increasing_id 
from math import sqrt
# from pyspark.sql.functions import sqrt

# Data
train_features = spark.read.csv('../Data/Train-28x28.csv', header=False, inferSchema="true")
test_features = spark.read.csv('../Data/Test-28x28.csv', header=False, inferSchema="true")
train_labels = spark.read.csv('../Data/Train-label.csv', header=False, inferSchema="true")
test_labels = spark.read.csv('../Data/Test-label.csv', header=False, inferSchema="true")

def get_vector(data, col_name):
    assembler = VectorAssembler(inputCols=data.columns, outputCol=col_name)
    return assembler.transform(data).select(col_name)    

# Vectors
train_features = get_vector(train_features, 'train_features')
test_features = get_vector(test_features, 'test_features')
train_labels = get_vector(train_labels, 'train_labels')
test_labels = get_vector(test_labels, 'test_labels')

def combine_features_labels(feature_vector, label_vector, kind='train'):
    features = feature_vector.withColumn('{}_id'.format(kind), monotonically_increasing_id())
    labels = label_vector.withColumn('{}_id'.format(kind), monotonically_increasing_id())
    data = features.join(labels, '{}_id'.format(kind))
    return data

# Create combined train / test data
train_data = combine_features_labels(train_features, train_labels, 'train')
test_data = combine_features_labels(test_features, test_labels, 'test')

In [2]:
# This is a cartesian join using dataframes
cross = train_data.crossJoin(test_data)
cross.show(3), cross.count()

+--------+--------------------+------------+-------+--------------------+-----------+
|train_id|      train_features|train_labels|test_id|       test_features|test_labels|
+--------+--------------------+------------+-------+--------------------+-----------+
|       0|(784,[152,153,154...|       [5.0]|      0|(784,[202,203,204...|      [7.0]|
|       0|(784,[152,153,154...|       [5.0]|      1|(784,[94,95,96,97...|      [2.0]|
|       0|(784,[152,153,154...|       [5.0]|      2|(784,[128,129,130...|      [1.0]|
+--------+--------------------+------------+-------+--------------------+-----------+
only showing top 3 rows



(None, 48307590)

In [98]:
from math import sqrt
from collections import Counter

def cosine_distance(vector_one, vector_two):
    dot_product = vector_one.dot(vector_two)
    sum_sq_vector_one = sqrt(vector_one.dot(vector_one))
    sum_sq_vector_two = sqrt(vector_two.dot(vector_two))
    return float(dot_product / (sum_sq_vector_one * sum_sq_vector_two))

def majority_vote(neighbors):
    closest_instance = neighbors[0][1]
    votes = [n[1] for n in neighbors]
    counted_votes = Counter(votes)
    most_common_vote = counted_votes.most_common(1)[0]
    if most_common_vote[1] > 1:
        return float(most_common_vote[0])
    else:
        return float(closest_instance)

In [97]:
K = 7

d = cross.rdd \
    .map(lambda x: (x.test_id, (cosine_distance(x.train_features, x.test_features), x.train_labels[0]))) \
    .groupByKey() \
    .map(lambda x: (x[0], majority_vote(sorted(list(x[1]))[:K]))) \
    .collect()

#     .map(lambda x: sorted(list(x[1]))[:K]) \
#     .collect()

KeyboardInterrupt: 

In [99]:
# t = cross.rdd.map(lambda x: x.test_features.dot(x.test_features)).collect()

In [106]:
d = cross.collect()

Py4JJavaError: An error occurred while calling o1189.collectToPython.
: org.apache.spark.SparkException: Job 98 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:837)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:835)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:835)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:1838)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:83)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:1751)
	at org.apache.spark.SparkContext$$anonfun$stop$8.apply$mcV$sp(SparkContext.scala:1924)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1357)
	at org.apache.spark.SparkContext.stop(SparkContext.scala:1923)
	at org.apache

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 60407)
Traceback (most recent call last):
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/usr/local/Cellar/apache-spark/2.3.0

In [ ]:
preds = sc.parallelize(d).toDF(['id', 'prediction'])

In [ ]:
preds.show(), test_data.show()

In [15]:
# d = cross.rdd \
#     .map(lambda x: (x.test_id, (x.train_labels[0], cosine_distance(x.train_features, x.test_features)))) \
#     .groupByKey() \
#     .map(lambda x: type(x[1])) \
#     .collect()
#     .map(lambda x: (x[0], sorted(x[1])[:K])) \
#     .map(lambda x: (x[0], majority_vote(x[1]))) \

In [ ]:
# .map(lambda x: (x[0].id, x[1].labels[0], cosine_distance(x[0].features, x[1].features)))

In [112]:
cross.show(500)

+--------+--------------------+------------+-------+--------------------+-----------+
|train_id|      train_features|train_labels|test_id|       test_features|test_labels|
+--------+--------------------+------------+-------+--------------------+-----------+
|       0|(784,[152,153,154...|       [5.0]|      0|(784,[202,203,204...|      [7.0]|
|       0|(784,[152,153,154...|       [5.0]|      1|(784,[94,95,96,97...|      [2.0]|
|       0|(784,[152,153,154...|       [5.0]|      2|(784,[128,129,130...|      [1.0]|
|       0|(784,[152,153,154...|       [5.0]|      3|(784,[124,125,126...|      [0.0]|
|       0|(784,[152,153,154...|       [5.0]|      4|(784,[150,151,159...|      [4.0]|
|       0|(784,[152,153,154...|       [5.0]|      5|(784,[156,157,158...|      [1.0]|
|       0|(784,[152,153,154...|       [5.0]|      6|(784,[149,150,151...|      [4.0]|
|       0|(784,[152,153,154...|       [5.0]|      7|(784,[179,180,181...|      [9.0]|
|       0|(784,[152,153,154...|       [5.0]|      8|(7

In [ ]:
# def majority_vote(neighbors):
#     closest_instance = neighbors[0][1]
#     votes = [n[1] for n in neighbors]
#     counted_votes = Counter(votes)
#     most_common_vote = counted_votes.most_common(1)[0]
#     if most_common_vote[1] > 1:
#         return most_common_vote[0]
#     else:
#         return closest_instance

# def cosine_distance(vector_one, vector_two):
#     dot_product = vector_one.dot(vector_two)
#     sum_sq_vector_one = sqrt(vector_one.dot(vector_one))
#     sum_sq_vector_two = sqrt(vector_two.dot(vector_two))
    
#     return dot_product / (sum_sq_vector_one * sum_sq_vector_two)

# K = 3

# d = test_data.rdd \
#     .cartesian(train_data.rdd) \
#     .map(lambda x: (x[0].id, x[1].labels[0], cosine_distance(x[0].features, x[1].features))) \
#     .map(lambda x: (x[0], (x[2], x[1]))) \
#     .groupByKey() \
#     .map(lambda x: (x[0], sorted(x[1])[:K])) \
#     .map(lambda x: (x[0], majority_vote(x[1]))) \

In [61]:
# import org.apache.spark.sql.functions._

# test_data.withColumn("dummy", explode(array((list(range(0, 10))).map(lit): _*)))
test_data.join(test_data, test_data.id == test_data.id).show()

+---+--------------------+------+---+--------------------+------+
| id|            features|labels| id|            features|labels|
+---+--------------------+------+---+--------------------+------+
|  0|(784,[202,203,204...| [7.0]|  0|(784,[202,203,204...| [7.0]|
|  1|(784,[94,95,96,97...| [2.0]|  1|(784,[94,95,96,97...| [2.0]|
|  2|(784,[128,129,130...| [1.0]|  2|(784,[128,129,130...| [1.0]|
|  3|(784,[124,125,126...| [0.0]|  3|(784,[124,125,126...| [0.0]|
|  4|(784,[150,151,159...| [4.0]|  4|(784,[150,151,159...| [4.0]|
|  5|(784,[156,157,158...| [1.0]|  5|(784,[156,157,158...| [1.0]|
|  6|(784,[149,150,151...| [4.0]|  6|(784,[149,150,151...| [4.0]|
|  7|(784,[179,180,181...| [9.0]|  7|(784,[179,180,181...| [9.0]|
|  8|(784,[129,130,131...| [5.0]|  8|(784,[129,130,131...| [5.0]|
|  9|(784,[209,210,211...| [9.0]|  9|(784,[209,210,211...| [9.0]|
| 10|(784,[123,124,125...| [0.0]| 10|(784,[123,124,125...| [0.0]|
| 11|(784,[94,95,96,97...| [6.0]| 11|(784,[94,95,96,97...| [6.0]|
| 12|(784,

In [53]:
cross.show()

AnalysisException: 'Detected cartesian product for INNER join between logical plans\nProject [id#16674L, features#15868, labels#16661]\n+- Join Inner, (id#16674L = id#16677L)\n   :- Project [features#15868, monotonically_increasing_id() AS id#16674L]\n   :  +- Project [UDF(named_struct(_c0_double_VectorAssembler_4e59bb9f2b36faee42a7, cast(_c0#11921 as double), _c1_double_VectorAssembler_4e59bb9f2b36faee42a7, cast(_c1#11922 as double), _c2_double_VectorAssembler_4e59bb9f2b36faee42a7, cast(_c2#11923 as double), _c3_double_VectorAssembler_4e59bb9f2b36faee42a7, cast(_c3#11924 as double), _c4_double_VectorAssembler_4e59bb9f2b36faee42a7, cast(_c4#11925 as double), _c5_double_VectorAssembler_4e59bb9f2b36faee42a7, cast(_c5#11926 as double), _c6_double_VectorAssembler_4e59bb9f2b36faee42a7, cast(_c6#11927 as double), _c7_double_VectorAssembler_4e59bb9f2b36faee42a7, cast(_c7#11928 as double), _c8_double_VectorAssembler_4e59bb9f2b36faee42a7, cast(_c8#11929 as double), _c9_double_VectorAssembler_4e

In [44]:
# test_data.rdd.map(lambda x: sqrt(x[1][100])).collect()

In [6]:
train_data.show(5)

+---+--------------------+------+
| id|            features|labels|
+---+--------------------+------+
|  0|(784,[152,153,154...| [5.0]|
|  1|(784,[127,128,129...| [0.0]|
|  2|(784,[160,161,162...| [4.0]|
|  3|(784,[158,159,160...| [1.0]|
|  4|(784,[208,209,210...| [9.0]|
+---+--------------------+------+
only showing top 5 rows



In [30]:
from collections import Counter
import numpy as np
from pyspark.sql.functions import sqrt
from pyspark.sql.types import DoubleType

def majority_vote(neighbors):
    closest_instance = neighbors[0][1]
    votes = [n[1] for n in neighbors]
    counted_votes = Counter(votes)
    most_common_vote = counted_votes.most_common(1)[0]
    if most_common_vote[1] > 1:
        return most_common_vote[0]
    else:
        return closest_instance

def cosine_distance(vector_one, vector_two):
    dot_product = vector_one.dot(vector_two)
    sum_sq_vector_one = sqrt(vector_one.dot(vector_one))
    sum_sq_vector_two = sqrt(vector_two.dot(vector_two))
    
    return dot_product / (sum_sq_vector_one * sum_sq_vector_two)

K = 3

d = test_data.rdd \
    .cartesian(train_data.rdd) \
    .map(lambda x: (x[0].id, x[1].labels[0], cosine_distance(x[0].features, x[1].features))) \
    .map(lambda x: (x[0], (x[2], x[1]))) \
    .groupByKey() \
    .map(lambda x: (x[0], sorted(x[1])[:K])) \
    .map(lambda x: (x[0], majority_vote(x[1]))) \

In [31]:
preds = d.collect() # 1:24

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 27.0 failed 1 times, most recent failure: Lost task 0.0 in stage 27.0 (TID 27, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 229, in main
    process()
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 224, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/rdd.py", line 2438, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/rdd.py", line 2438, in pipeline_func
    return func(split, prev_func(split, iterator))
 

In [32]:
preds

[(0, 1.0),
 (1, 6.0),
 (2, 7.0),
 (3, 1.0),
 (4, 9.0),
 (5, 4.0),
 (6, 1.0),
 (7, 0.0),
 (8, 5.0),
 (9, 0.0),
 (10, 9.0),
 (11, 0.0),
 (12, 0.0),
 (13, 6.0),
 (14, 4.0),
 (15, 5.0),
 (16, 1.0),
 (17, 1.0),
 (18, 2.0),
 (19, 1.0),
 (20, 0.0),
 (21, 0.0),
 (22, 0.0),
 (23, 7.0),
 (24, 1.0),
 (25, 1.0),
 (26, 1.0),
 (27, 1.0),
 (28, 6.0),
 (29, 4.0),
 (30, 7.0),
 (31, 4.0),
 (32, 6.0),
 (33, 1.0),
 (34, 7.0),
 (35, 2.0),
 (36, 1.0),
 (37, 4.0),
 (38, 4.0),
 (39, 4.0),
 (40, 7.0),
 (41, 5.0),
 (42, 4.0),
 (43, 7.0),
 (44, 7.0),
 (45, 7.0),
 (46, 7.0),
 (47, 7.0),
 (48, 1.0),
 (49, 1.0),
 (50, 0.0),
 (51, 6.0),
 (52, 6.0),
 (53, 6.0),
 (54, 5.0),
 (55, 4.0),
 (56, 1.0),
 (57, 4.0),
 (58, 0.0),
 (59, 6.0),
 (60, 5.0),
 (61, 7.0),
 (62, 7.0),
 (63, 7.0),
 (64, 7.0),
 (65, 0.0),
 (66, 0.0),
 (67, 1.0),
 (68, 4.0),
 (69, 7.0),
 (70, 1.0),
 (71, 6.0),
 (72, 9.0),
 (73, 4.0),
 (74, 4.0),
 (75, 0.0),
 (76, 4.0),
 (77, 0.0),
 (78, 6.0),
 (79, 4.0),
 (80, 4.0),
 (81, 1.0),
 (82, 7.0),
 (83, 1.0),
 (

In [34]:
sc.parallelize(preds).collect()

[(0, 1.0),
 (1, 6.0),
 (2, 7.0),
 (3, 1.0),
 (4, 9.0),
 (5, 4.0),
 (6, 1.0),
 (7, 0.0),
 (8, 5.0),
 (9, 0.0),
 (10, 9.0),
 (11, 0.0),
 (12, 0.0),
 (13, 6.0),
 (14, 4.0),
 (15, 5.0),
 (16, 1.0),
 (17, 1.0),
 (18, 2.0),
 (19, 1.0),
 (20, 0.0),
 (21, 0.0),
 (22, 0.0),
 (23, 7.0),
 (24, 1.0),
 (25, 1.0),
 (26, 1.0),
 (27, 1.0),
 (28, 6.0),
 (29, 4.0),
 (30, 7.0),
 (31, 4.0),
 (32, 6.0),
 (33, 1.0),
 (34, 7.0),
 (35, 2.0),
 (36, 1.0),
 (37, 4.0),
 (38, 4.0),
 (39, 4.0),
 (40, 7.0),
 (41, 5.0),
 (42, 4.0),
 (43, 7.0),
 (44, 7.0),
 (45, 7.0),
 (46, 7.0),
 (47, 7.0),
 (48, 1.0),
 (49, 1.0),
 (50, 0.0),
 (51, 6.0),
 (52, 6.0),
 (53, 6.0),
 (54, 5.0),
 (55, 4.0),
 (56, 1.0),
 (57, 4.0),
 (58, 0.0),
 (59, 6.0),
 (60, 5.0),
 (61, 7.0),
 (62, 7.0),
 (63, 7.0),
 (64, 7.0),
 (65, 0.0),
 (66, 0.0),
 (67, 1.0),
 (68, 4.0),
 (69, 7.0),
 (70, 1.0),
 (71, 6.0),
 (72, 9.0),
 (73, 4.0),
 (74, 4.0),
 (75, 0.0),
 (76, 4.0),
 (77, 0.0),
 (78, 6.0),
 (79, 4.0),
 (80, 4.0),
 (81, 1.0),
 (82, 7.0),
 (83, 1.0),
 (

In [28]:
# spark.createDataFrame(preds, ['id', 'prediction'])

TypeError: not supported type: <class 'numpy.float64'>

In [17]:
spark.createDataFrame(preds)

TypeError: Can not infer schema for type: <class 'numpy.float64'>

In [14]:
# def evaluate()

[7.0,
 2.0,
 1.0,
 0.0,
 4.0,
 1.0,
 4.0,
 9.0,
 5.0,
 9.0,
 0.0,
 6.0,
 9.0,
 0.0,
 1.0,
 5.0,
 9.0,
 7.0,
 3.0,
 4.0,
 9.0,
 6.0,
 6.0,
 5.0,
 4.0,
 0.0,
 7.0,
 4.0,
 0.0,
 1.0,
 3.0,
 1.0,
 3.0,
 4.0,
 7.0,
 2.0,
 7.0,
 1.0,
 2.0,
 1.0,
 1.0,
 7.0,
 4.0,
 2.0,
 3.0,
 5.0,
 1.0,
 2.0,
 4.0,
 4.0,
 6.0,
 3.0,
 5.0,
 5.0,
 6.0,
 0.0,
 4.0,
 1.0,
 9.0,
 5.0,
 7.0,
 8.0,
 9.0,
 3.0,
 7.0,
 4.0,
 6.0,
 4.0,
 3.0,
 0.0,
 7.0,
 0.0,
 2.0,
 9.0,
 1.0,
 7.0,
 3.0,
 2.0,
 9.0,
 7.0,
 7.0,
 6.0,
 2.0,
 7.0,
 8.0,
 4.0,
 7.0,
 3.0,
 6.0,
 1.0,
 3.0,
 6.0,
 9.0,
 3.0,
 1.0,
 4.0,
 1.0,
 7.0,
 6.0,
 9.0]

In [10]:
type(preds)

list

In [575]:
# test_data.show()

In [297]:
train_features

DataFrame[features: vector]

In [282]:
train_features.show(3), train_lables.show(3)

+--------------------+
|            features|
+--------------------+
|(784,[152,153,154...|
|(784,[127,128,129...|
|(784,[160,161,162...|
+--------------------+
only showing top 3 rows

+------+
|labels|
+------+
| [5.0]|
| [0.0]|
| [4.0]|
+------+
only showing top 3 rows



(None, None)

In [400]:
from pyspark.sql.functions import monotonically_increasing_id 

# This will return a new DF with all the columns + id
train_features = train_features.withColumn("id", monotonically_increasing_id())
train_labels = train_labels.withColumn("id", monotonically_increasing_id())
train_data = train_features.join(train_labels, "id")

train_data.show()

+---+--------------------+------+
| id|            features|labels|
+---+--------------------+------+
|  0|(784,[152,153,154...| [5.0]|
|  1|(784,[127,128,129...| [0.0]|
|  2|(784,[160,161,162...| [4.0]|
|  3|(784,[158,159,160...| [1.0]|
|  4|(784,[208,209,210...| [9.0]|
|  5|(784,[155,156,157...| [2.0]|
|  6|(784,[124,125,126...| [1.0]|
|  7|(784,[151,152,153...| [3.0]|
|  8|(784,[152,153,154...| [1.0]|
|  9|(784,[134,135,161...| [4.0]|
| 10|(784,[123,124,125...| [3.0]|
| 11|(784,[216,217,218...| [5.0]|
| 12|(784,[143,144,145...| [3.0]|
| 13|(784,[72,73,74,99...| [6.0]|
| 14|(784,[151,152,153...| [1.0]|
| 15|(784,[211,212,213...| [7.0]|
| 16|(784,[151,152,153...| [2.0]|
| 17|(784,[159,160,161...| [8.0]|
| 18|(784,[100,101,102...| [6.0]|
| 19|(784,[209,210,211...| [9.0]|
+---+--------------------+------+



In [429]:
# 0 - test instance, 1- train instance

# test_data.rdd.cartesian(train_data.rdd).map(lambda x : x[0].id).collect() # get all test ids
# test_data.rdd.cartesian(train_data.rdd).map(lambda x : x[1].id).collect() # get all train ids
# test_data.rdd.cartesian(train_data.rdd).map(lambda x : x[0].features).collect() # get all test features (grouping train)
# test_data.rdd.cartesian(train_data.rdd).map(lambda x : x[1].features).collect() # get all train features (grouping train)
# test_data.rdd.cartesian(train_data.rdd).map(lambda x : x[0].labels[0]).collect() # get all test lables (grouping train)
# test_data.rdd.cartesian(train_data.rdd).map(lambda x : x[0].labels[1]).collect() # get all train lables (grouping train)

In [460]:
test_data.show()

+---+--------------------+------+
| id|            features|labels|
+---+--------------------+------+
|  0|(784,[202,203,204...| [7.0]|
|  1|(784,[94,95,96,97...| [2.0]|
|  2|(784,[128,129,130...| [1.0]|
|  3|(784,[124,125,126...| [0.0]|
|  4|(784,[150,151,159...| [4.0]|
+---+--------------------+------+



In [570]:
# .map(lambda x: (x[0][1], cosine_distance(x[0][0][0], x[1][0])))

# d = test_data.rdd.cartesian(train_data.rdd).map(lambda x: (x[0].id, x[1].label, cosine_distance(x[0].features, x[1].features)))

# neighbors - list
# neighbors[0] - tuple
from collections import Counter

def majority_vote(neighbors):
    closest_instance = neighbors[0][1]
    votes = [n[1] for n in neighbors]
    counted_votes = Counter(votes)
    most_common_vote = counted_votes.most_common(1)[0]
    if most_common_vote[1] > 1:
        return most_common_vote[0]
    else:
        return closest_instance

K = 3

d = test_data.rdd \
    .cartesian(train_data.rdd) \
    .map(lambda x: (x[0].id, x[1].labels[0], cosine_distance(x[0].features, x[1].features))) \
    .map(lambda x: (x[0], (x[2], x[1]))) \
    .groupByKey() \
    .map(lambda x: (x[0], sorted(x[1])[:K])) \
    .map(lambda x: (x[0], majority_vote(x[1]))) \
    .map(lambda x: x[1])

d.collect()

[1.0, 4.0, 4.0, 1.0, 1.0]

In [430]:
# from pyspark.sql.functions import struct
# train_features.withColumn('train', train_lables.labels).show()

In [213]:
def cartesian_product(test):
    return train_features.rdd.cartesian(test)

cp = test_features.rdd.map(lambda x: print(x))
cp.collect()

[None, None, None, None, None]

In [331]:
def cosine_distance(vector_one, vector_two):
    dot_product = vector_one.dot(vector_two)
    sum_sq_vector_one = np.sqrt(vector_one.dot(vector_one))
    sum_sq_vector_two = np.sqrt(vector_two.dot(vector_two))
    
    return dot_product / (sum_sq_vector_one * sum_sq_vector_two)

In [363]:
# This is all pairs indexed by testing example

# .map(lambda x: x[0][1]) - returns testing index; .map(lambda x: x[0]) // .map(lambda x: x[1]) - returns Row

# x[0][1] - test idx
# x[0][0][0] - set of sparse vectors
# x[1][0] - second set of sparse vectors

t = test_features.rdd.zipWithIndex().cartesian(train_features.rdd)\
        .map(lambda x: (x[0][1], cosine_distance(x[0][0][0], x[1][0])))
t.collect()

[(0, 0.4233402126471753),
 (0, 0.3504498647140426),
 (0, 0.2475203701546623),
 (0, 0.14505681521036096),
 (0, 0.3833998199165904),
 (0, 0.41000320202928575),
 (0, 0.28028561364776344),
 (0, 0.43703378439042845),
 (0, 0.2457241788293801),
 (0, 0.3957665835774502),
 (0, 0.42046162436678675),
 (0, 0.1588951918748992),
 (0, 0.27749083062558744),
 (0, 0.28621629373349766),
 (0, 0.25145865663477807),
 (0, 0.5810224704170716),
 (0, 0.432374854463215),
 (0, 0.36488175266179246),
 (0, 0.17683650799522269),
 (0, 0.41663645683785683),
 (1, 0.3367887124710152),
 (2, 0.349082892881825),
 (1, 0.34776410312842115),
 (1, 0.13543287821558847),
 (2, 0.29210832215141075),
 (2, 0.07255649507447434),
 (1, 0.2516065622777885),
 (1, 0.23853337566027735),
 (1, 0.3095924163625588),
 (1, 0.36114583763649794),
 (2, 0.5442291885374303),
 (2, 0.2765991725703911),
 (2, 0.27354937051008343),
 (2, 0.38751558760046906),
 (1, 0.34286474576464304),
 (1, 0.27677852465141756),
 (1, 0.25021497179997126),
 (1, 0.50908636090

In [303]:
from pyspark.sql.functions import sqrt
import numpy as np

# dot_product = test_features.rdd.cartesian(train_features.rdd) \
#                 .map(lambda x: x[0][0].dot(x[1][0]))

# dot_product = test_features.rdd.cartesian(train_features.rdd) \
#                 .map(lambda x: x[0][0].dot(x[0][0]) / x[0][0].dot(x[0][0]))

cosine_dist = test_features.rdd.cartesian(train_features.rdd) \
                .map(lambda x: cosine_distance(x[0][0], x[1][0]))

n_test = test_features.count()
n_train = train_features.count()

np_dists = np.array(cosine_dist.collect()).reshape(n_test, n_train).tolist()
np_sc = sc.parallelize(np_dists)
np_sc.collect()

[[0.4233402126471753,
  0.3504498647140426,
  0.2475203701546623,
  0.14505681521036096,
  0.3833998199165904,
  0.41000320202928575,
  0.28028561364776344,
  0.43703378439042845,
  0.2457241788293801,
  0.3957665835774502,
  0.42046162436678675,
  0.1588951918748992,
  0.27749083062558744,
  0.28621629373349766,
  0.25145865663477807,
  0.5810224704170716,
  0.432374854463215,
  0.36488175266179246,
  0.17683650799522269,
  0.41663645683785683],
 [0.3367887124710152,
  0.349082892881825,
  0.34776410312842115,
  0.13543287821558847,
  0.29210832215141075,
  0.07255649507447434,
  0.2516065622777885,
  0.23853337566027735,
  0.3095924163625588,
  0.36114583763649794,
  0.5442291885374303,
  0.2765991725703911,
  0.27354937051008343,
  0.38751558760046906,
  0.34286474576464304,
  0.27677852465141756,
  0.25021497179997126,
  0.509086360907887,
  0.28478752013261877,
  0.48794158504482055],
 [0.34878686752408716,
  0.31044732519857615,
  0.3081816890666162,
  0.3417621038910506,
  0.323

In [185]:
t = train_features.rdd.cartesian(train_features.rdd).count()
t

400

In [106]:
# s = test_features.rdd.foreach(lambda x: train_features.rdd.foreach(lambda y : print(x, y)))

In [98]:
t = train_features.rdd.map(lambda x: x[0])
t.collect()

[SparseVector(784, {152: 3.0, 153: 18.0, 154: 18.0, 155: 18.0, 156: 126.0, 157: 136.0, 158: 175.0, 159: 26.0, 160: 166.0, 161: 255.0, 162: 247.0, 163: 127.0, 176: 30.0, 177: 36.0, 178: 94.0, 179: 154.0, 180: 170.0, 181: 253.0, 182: 253.0, 183: 253.0, 184: 253.0, 185: 253.0, 186: 225.0, 187: 172.0, 188: 253.0, 189: 242.0, 190: 195.0, 191: 64.0, 203: 49.0, 204: 238.0, 205: 253.0, 206: 253.0, 207: 253.0, 208: 253.0, 209: 253.0, 210: 253.0, 211: 253.0, 212: 253.0, 213: 251.0, 214: 93.0, 215: 82.0, 216: 82.0, 217: 56.0, 218: 39.0, 231: 18.0, 232: 219.0, 233: 253.0, 234: 253.0, 235: 253.0, 236: 253.0, 237: 253.0, 238: 198.0, 239: 182.0, 240: 247.0, 241: 241.0, 260: 80.0, 261: 156.0, 262: 107.0, 263: 253.0, 264: 253.0, 265: 205.0, 266: 11.0, 268: 43.0, 269: 154.0, 289: 14.0, 290: 1.0, 291: 154.0, 292: 253.0, 293: 90.0, 319: 139.0, 320: 253.0, 321: 190.0, 322: 2.0, 347: 11.0, 348: 190.0, 349: 253.0, 350: 70.0, 376: 35.0, 377: 241.0, 378: 225.0, 379: 160.0, 380: 108.0, 381: 1.0, 405: 81.0, 406:

In [84]:
train_features.rdd.f, test_features.count()

AttributeError: 'RDD' object has no attribute 'dot'

In [38]:
t = train_features.rdd.cartesian(train_features.rdd).map(lambda x: x[0][0])
t.collect()

[SparseVector(784, {152: 3.0, 153: 18.0, 154: 18.0, 155: 18.0, 156: 126.0, 157: 136.0, 158: 175.0, 159: 26.0, 160: 166.0, 161: 255.0, 162: 247.0, 163: 127.0, 176: 30.0, 177: 36.0, 178: 94.0, 179: 154.0, 180: 170.0, 181: 253.0, 182: 253.0, 183: 253.0, 184: 253.0, 185: 253.0, 186: 225.0, 187: 172.0, 188: 253.0, 189: 242.0, 190: 195.0, 191: 64.0, 203: 49.0, 204: 238.0, 205: 253.0, 206: 253.0, 207: 253.0, 208: 253.0, 209: 253.0, 210: 253.0, 211: 253.0, 212: 253.0, 213: 251.0, 214: 93.0, 215: 82.0, 216: 82.0, 217: 56.0, 218: 39.0, 231: 18.0, 232: 219.0, 233: 253.0, 234: 253.0, 235: 253.0, 236: 253.0, 237: 253.0, 238: 198.0, 239: 182.0, 240: 247.0, 241: 241.0, 260: 80.0, 261: 156.0, 262: 107.0, 263: 253.0, 264: 253.0, 265: 205.0, 266: 11.0, 268: 43.0, 269: 154.0, 289: 14.0, 290: 1.0, 291: 154.0, 292: 253.0, 293: 90.0, 319: 139.0, 320: 253.0, 321: 190.0, 322: 2.0, 347: 11.0, 348: 190.0, 349: 253.0, 350: 70.0, 376: 35.0, 377: 241.0, 378: 225.0, 379: 160.0, 380: 108.0, 381: 1.0, 405: 81.0, 406:

In [ ]:
def dot_product(vector_one, vector_two):
    catresian_product = vector_one.cartesian(vector_two) \
    .filter(lambda x: x[0][0] != x[1][0]) \
    

In [ ]:
def computeDot(sparseVectorA, sparseVectorB):
    """
    Function to compute dot product of two SparseVectors
    """
    return sparseVectorA.dot(sparseVectorB)

# Use Cartesian function on the RDD to create tuples containing 
# 2-combinations of all the rows in the original RDD

combinationRDD = (originalRDD.cartesian(originalRDD))

# The records in combinationRDD will be of the form 
# [(Index, SV1), (Index, SV1)], therefore, you need to
# filter all the records where the index is not equal giving
# RDD of the form [(Index1, SV1), (Index2, SV2)] and so on,
# then use the map function to use the SparseVector's dot function

dottedRDD = (combinationRDD
             .filter(lambda x: x[0][0] != x[1][0])
             .map(lambda x: computeDot(x[0][1], x[1][1])
             .cache())

In [4]:
train_features.rdd.map(lambda row: row.features).count()

20

In [152]:
test_features.rdd.foreach(lambda t:t*2).collect()

In [68]:
knn = test_features.rdd.cartesian(train_features.rdd)

In [207]:
spark.createDataFrame(knn).show()

+--------------------+--------------------+
|                  _1|                  _2|
+--------------------+--------------------+
|[(784,[202,203,20...|[(784,[152,153,15...|
|[(784,[202,203,20...|[(784,[127,128,12...|
|[(784,[202,203,20...|[(784,[160,161,16...|
|[(784,[202,203,20...|[(784,[158,159,16...|
|[(784,[202,203,20...|[(784,[208,209,21...|
|[(784,[202,203,20...|[(784,[155,156,15...|
|[(784,[202,203,20...|[(784,[124,125,12...|
|[(784,[202,203,20...|[(784,[151,152,15...|
|[(784,[202,203,20...|[(784,[152,153,15...|
|[(784,[202,203,20...|[(784,[134,135,16...|
|[(784,[202,203,20...|[(784,[123,124,12...|
|[(784,[202,203,20...|[(784,[216,217,21...|
|[(784,[202,203,20...|[(784,[143,144,14...|
|[(784,[202,203,20...|[(784,[72,73,74,9...|
|[(784,[202,203,20...|[(784,[151,152,15...|
|[(784,[202,203,20...|[(784,[211,212,21...|
|[(784,[202,203,20...|[(784,[151,152,15...|
|[(784,[202,203,20...|[(784,[159,160,16...|
|[(784,[202,203,20...|[(784,[100,101,10...|
|[(784,[202,203,20...|[(784,[209

In [209]:
t = knn.map(lambda x: x[0] * x[1])

In [210]:
t.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 138.0 failed 1 times, most recent failure: Lost task 0.0 in stage 138.0 (TID 138, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 229, in main
    process()
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 224, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/lib/pyspark.zip/pyspark/serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-209-03454bd83aab>", line 1, in <lambda>
TypeError: can't multiply sequence by non-int of type 'Row'

	at org.apa

In [93]:
spark.createDataFrame(knn).show(3)

+--------------------+--------------------+
|                  _1|                  _2|
+--------------------+--------------------+
|[(784,[202,203,20...|[(784,[152,153,15...|
|[(784,[202,203,20...|[(784,[127,128,12...|
|[(784,[202,203,20...|[(784,[160,161,16...|
+--------------------+--------------------+
only showing top 3 rows



In [112]:
spark.createDataFrame(knn)[0]

Column<b'_1'>

In [84]:
spark.createDataFrame(knn)[0].dot(spark.createDataFrame(knn)[1])

TypeError: 'Column' object is not callable

In [76]:
spark.createDataFrame(knn.map(lambda x: x[0].features * x[1].features)).show()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 40.0 failed 1 times, most recent failure: Lost task 0.0 in stage 40.0 (TID 40, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 229, in main
    process()
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 224, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/lib/pyspark.zip/pyspark/serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/rdd.py", line 1354, in takeUpToNumLeft
    yield next(iterator)
  File "<ipytho

In [7]:
from pyspark.sql.functions import sqrt

def sum_sq(vector):
    return vector.map(lambda x: x * x).sum()

def dot(vector_one, vector_two):
    return vector_one.dot(vector_two)

def cosine(vector_one, vector_two):
    return dot(vector_one, vector_two) / (sqrt(sum_sq(vector_one)) * sqrt(sum_sq(vector_two)))

In [ ]:
knn = test_features.rdd.cartesian(train_features.rdd)

In [11]:
train_features

DataFrame[features: vector]

In [27]:
train_features.rdd.count(), test_features.rdd.count()

(20, 5)

In [30]:
knn = test_features.rdd.cartesian(train_features.rdd)

In [36]:
knn = test_features.foreach(lambda x: x.rdd.cartesian(train_features.rdd))

Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/cloudpickle.py", line 235, in dump
    return Pickler.dump(self, obj)
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/pickle.py", line 409, in dump
    self.save(obj)
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/pickle.py", line 751, in save_tuple
    save(element)
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/cloudpickle.py", line 378, in save_function
    self.save_function_tuple(obj)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/cloudpickle.py", line 529, in save_fun

PicklingError: Could not serialize object: Py4JError: An error occurred while calling o241.__getstate__. Trace:
py4j.Py4JException: Method __getstate__([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:745)



In [32]:
knn.count()

Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/cloudpickle.py", line 235, in dump
    return Pickler.dump(self, obj)
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/pickle.py", line 409, in dump
    self.save(obj)
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/pickle.py", line 751, in save_tuple
    save(element)
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/cloudpickle.py", line 378, in save_function
    self.save_function_tuple(obj)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/cloudpickle.py", line 529, in save_fun

PicklingError: Could not serialize object: Py4JError: An error occurred while calling o241.__getstate__. Trace:
py4j.Py4JException: Method __getstate__([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:745)



In [1]:
assembler = VectorAssembler(inputCols=train_features.columns, outputCol="features")
test_vectors = assembler.transform(train_features).select("features")
test_vectors.head(2)

NameError: name 'VectorAssembler' is not defined

In [11]:
import numpy as np; np.array(test_vectors.collect()).shape

KeyboardInterrupt: 

Error in atexit._run_exitfuncs:
Traceback (most recent call last):
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/threading.py", line 551, in wait
    signaled = self._cond.wait(timeout)
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/threading.py", line 295, in wait
    waiter.acquire()
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/context.py", line 233, in signal_handler
    self.cancelAllJobs()
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/context.py", line 971, in cancelAllJobs
    self._jsc.sc().cancelAllJobs()
AttributeError: 'NoneType' object has no attribute 'sc'


In [4]:
from pyspark.ml.feature import PCA

pca = PCA(k=153, inputCol="features", outputCol="pcaFeatures")
pca_model = pca.fit(train_vectors)

# Apply PCA to train / test features
train_vectors_pca = pca_model.transform(train_vectors).select("pcaFeatures")
# map(testInstance => pcaTrain.cartesian(testInstance))
test_vectors_pca = pca_model.transform(test_vectors).select("pcaFeatures")

In [17]:
def find_distance(test_instance):
    distance = train_vectors_pca.rdd.cartesian(test_instance)
    return distance

In [27]:
knn = test_vectors_pca.rdd.cartesian(train_vectors_pca.rdd)

In [29]:
import numpy as np

result = np.array(knn.collect())
result.shape

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job 21 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:837)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:835)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:835)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:1838)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:83)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:1751)
	at org.apache.spark.SparkContext$$anonfun$stop$8.apply$mcV$sp(SparkContext.scala:1924)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1357)
	at org.apache.spark.SparkContext.stop(Spark

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 56237)
Traceback (most recent call last):
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/usr/local/Cellar/apache-spark/2.3.0

In [5]:
knn = train_vectors_pca.rdd.cartesian(test_vectors_pca.rdd).collect()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:55762)
Traceback (most recent call last):
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-7b2b679d4185>", line 1, in <module>
    train_vectors_pca.rdd.cartesian(test_vectors_pca.rdd).collect()
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/rdd.py", line 824, in collect
    port = self.ctx._jvm.PythonRDD.collectAndServe(self._jrdd.rdd())
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/lib/py4j-0.10.6-src.zip/py4j/java_gateway.py", line 1160, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:55762)
Traceback (most recent call last):
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-7b2b679d4185>", line 1, in <module>
    train_vectors_pca.rdd.cartesian(test_vectors_pca.rdd).collect()
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/rdd.py", line 824, in collect
    port = self.ctx._jvm.PythonRDD.collectAndServe(self._jrdd.rdd())
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/lib/py4j-0.10.6-src.zip/py4j/java_gateway.py", line 1160, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:55762)
Traceback (most recent call last):
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-7b2b679d4185>", line 1, in <module>
    train_vectors_pca.rdd.cartesian(test_vectors_pca.rdd).collect()
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/rdd.py", line 824, in collect
    port = self.ctx._jvm.PythonRDD.collectAndServe(self._jrdd.rdd())
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/lib/py4j-0.10.6-src.zip/py4j/java_gateway.py", line 1160, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:55762)
Traceback (most recent call last):
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-7b2b679d4185>", line 1, in <module>
    train_vectors_pca.rdd.cartesian(test_vectors_pca.rdd).collect()
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/rdd.py", line 824, in collect
    port = self.ctx._jvm.PythonRDD.collectAndServe(self._jrdd.rdd())
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/lib/py4j-0.10.6-src.zip/py4j/java_gateway.py", line 1160, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:55762)
Traceback (most recent call last):
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-7b2b679d4185>", line 1, in <module>
    train_vectors_pca.rdd.cartesian(test_vectors_pca.rdd).collect()
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/rdd.py", line 824, in collect
    port = self.ctx._jvm.PythonRDD.collectAndServe(self._jrdd.rdd())
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/lib/py4j-0.10.6-src.zip/py4j/java_gateway.py", line 1160, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/

...

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:55762)
Traceback (most recent call last):
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-7b2b679d4185>", line 1, in <module>
    train_vectors_pca.rdd.cartesian(test_vectors_pca.rdd).collect()
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/rdd.py", line 824, in collect
    port = self.ctx._jvm.PythonRDD.collectAndServe(self._jrdd.rdd())
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/lib/py4j-0.10.6-src.zip/py4j/java_gateway.py", line 1160, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:55762)
Traceback (most recent call last):
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-7b2b679d4185>", line 1, in <module>
    train_vectors_pca.rdd.cartesian(test_vectors_pca.rdd).collect()
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/rdd.py", line 824, in collect
    port = self.ctx._jvm.PythonRDD.collectAndServe(self._jrdd.rdd())
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/lib/py4j-0.10.6-src.zip/py4j/java_gateway.py", line 1160, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:55762)
Traceback (most recent call last):
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-7b2b679d4185>", line 1, in <module>
    train_vectors_pca.rdd.cartesian(test_vectors_pca.rdd).collect()
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/rdd.py", line 824, in collect
    port = self.ctx._jvm.PythonRDD.collectAndServe(self._jrdd.rdd())
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/lib/py4j-0.10.6-src.zip/py4j/java_gateway.py", line 1160, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:55762)
Traceback (most recent call last):
  File "/Users/nicholasmoore/Development/Anaconda/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-7b2b679d4185>", line 1, in <module>
    train_vectors_pca.rdd.cartesian(test_vectors_pca.rdd).collect()
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/rdd.py", line 824, in collect
    port = self.ctx._jvm.PythonRDD.collectAndServe(self._jrdd.rdd())
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/lib/py4j-0.10.6-src.zip/py4j/java_gateway.py", line 1160, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/usr/local/Cellar/apache-spark/2.3.0/libexec/

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:55762)